In [1]:
import numpy as np
import pandas as pd
import os

import anndata as ad
import scanpy as sc
import squidpy as sq

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors

sc.logging.print_header()
print(f"squidpy=={sq.__version__}")

scanpy==1.9.3 anndata==0.9.1 umap==0.5.3 numpy==1.22.4 scipy==1.9.1 pandas==1.5.0 scikit-learn==1.2.2 statsmodels==0.14.0 pynndescent==0.5.10
squidpy==1.2.3


In [2]:
# trigger core fonts for PDF backend
plt.rcParams["pdf.use14corefonts"] = True
# trigger core fonts for PS backend
plt.rcParams["ps.useafm"] = True

plt.rcParams['font.family'] = 'Sans serif'

### Step 0. Define color palette

In [3]:
value_palette = ["#FFFF00",
    "#a7c957",
    "#6a994e",
    "#90e0ef",
    "#0096c7",
    "#800080",
    "#A95C68",
    "#BF8F00",
    "#ff5a5f",
    "#AD002A",
    "#495057",
    "#ced4da"]
    
name_palette = ["Glomeruli",
    "PT",
    "Inj-T",
    "LH-CD",
    "Cast-T",
    "TLS",
    "I-IFTA",
    "Capsule",
    "Artery-C",
    "Artery-K",
    "Tumor",
    "Other"]
color_palette = dict(zip(name_palette, value_palette))


value_palette_glom = ["#fff088", 
                         "#c3ab00",
                         "#615500"]
names_palette_glom = ["DKDG",
                             "SGS",
                             "GGS"]
color_palette_glom = dict(zip(names_palette_glom, value_palette_glom))

palette_merge = {**color_palette, **color_palette_glom}

### Step 1. Load the dkd1 dataset

In [4]:
adata = sq.read.visium("../../data/AJHM365_dkd/outs/", source_image_path = "../../data/AJHM365_dkd/outs/spatial/tissue_hires_image.png")
adata.var_names_make_unique()

/home/users/dianli/.local/lib/python3.8/site-packages/anndata/_core/anndata.py:1832: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


#### Step 1.2. Load meta data

In [5]:
meta_morph_based = pd.read_csv("../../meta/Morph-based_v2.csv")
meta_morph_based.set_index('Barcode', inplace=True)
meta_morph_based = meta_morph_based.loc[adata.obs.index, ]
print(meta_morph_based.head(5))
print(meta_morph_based.shape)

                   Morph-based
AACACTTGGCAAGGAA-1       Tumor
AACAGGATTCATAGTT-1   Glomeruli
AACAGGTTATTGCACC-1          PT
AACAGGTTCACCGAAG-1       LH-CD
AACAGTCAGGCTCCGC-1       Other
(2802, 1)


In [6]:
adata.obs["Morph_based"] = meta_morph_based["Morph-based"]

In [7]:
adata.bk = adata.copy()

#### Step 1.3. Load glom meta data

In [8]:
meta_glom = pd.read_csv("../../meta/spots_selection - dkd_glom_v2_1.csv")
meta_glom.set_index('Barcode', inplace=True)
print(meta_glom.shape)
meta_glom.head()

(345, 2)


,Glomeruli_ID,Glom classification
Barcode,,
CATCGGCATGTCGAAG-1,G2,DKDG
CCATGAACAATGCTCC-1,G2,DKDG
TGTACCACATACTGGT-1,G2,DKDG
CGAGCGTAATGGAGTG-1,G3,DKDG
CTGTGGTAAGCTAATA-1,G3,DKDG


### Step 2. Create helper function for co-occurrence analysis

In [9]:
def co_occurrence_plot(adata = adata, 
                       meta_morph_based = meta_morph_based, 
                       meta_glom = meta_glom,
                       cluster_compare="PT", 
                       color_palette = palette_merge,
                       plot_path = "../../plots/version_1/main_figures/",
                       figsize = (6, 4), 
                       dpi = 300):
    plot_path = os.path.join(plot_path, "squidpy_co_occurrence")
    #### Generate output folder
    if not os.path.exists(plot_path):
        os.makedirs(plot_path)
    
    #### Update glom subcluter meta info
    meta_glom_CastT = meta_morph_based.copy()
    meta_glom_CastT["glom_sub"] = meta_glom_CastT["Morph-based"]

    meta_glom_CastT.loc[meta_glom_CastT['glom_sub'] != cluster_compare, 'glom_sub'] = 'Other'
    meta_glom_CastT.loc[meta_glom.index, 'glom_sub'] = meta_glom['Glom classification']
    
    adata.obs["glom_sub"] = meta_glom_CastT["glom_sub"].astype('category')
    #### Subset adata
    mask = ~adata.obs['glom_sub'].isin(['Other', 'SGS'])
    adata_subset = adata[mask].copy()
    
    #### Set color
    my_color = [color_palette[item] for item in sorted(["DKDG", "GGS", cluster_compare], key=str.casefold)]
    adata_subset.uns['glom_sub_colors'] = my_color
    
    ### Co_occurrence analysis
    sq.gr.co_occurrence(
        adata_subset,
        cluster_key="glom_sub",
    )
    
    ### generate and export plots
    sq.pl.co_occurrence(
        adata_subset,
        cluster_key="glom_sub",
        clusters=cluster_compare,
        figsize=(10, 10)
    )

    plt.xlabel('Distance', fontsize=12)
    plt.ylabel('Co-occurrence Score', fontsize=12)

    # Save the plot as PDF
    plt.gcf().set_size_inches(figsize)
    plt.savefig(os.path.join(plot_path, 'squidpy_co_occurrence_'+cluster_compare+'.pdf'), dpi=dpi, bbox_inches='tight')

    # Save the same plot as PNG
    plt.gcf().set_size_inches(figsize)
    plt.savefig(os.path.join(plot_path, 'squidpy_co_occurrence_'+cluster_compare+'.png'), dpi=dpi, bbox_inches='tight')

    # Clear the current figure after saving, if you're going to create more plots
    plt.clf()
    

In [10]:
for cluster_compare in ["PT", "Inj-T", "LH-CD", "Artery-K", "Cast-T", "TLS"]:
    co_occurrence_plot(cluster_compare=cluster_compare, plot_path = "../../plots/version_1/main_figures/")

  0%|          | 0/1 [00:00<?, ?/s]

  0%|          | 0/1 [00:00<?, ?/s]

  0%|          | 0/1 [00:00<?, ?/s]

  0%|          | 0/1 [00:00<?, ?/s]

  0%|          | 0/1 [00:00<?, ?/s]

...

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>